In [34]:
import requests
from bs4 import BeautifulSoup
from itertools import count

In [19]:
url_topicos = 'https://www.elnuevodia.com/topicos/seguridad/'
url_noticias= "https://www.elnuevodia.com/noticias/seguridad"
html = requests.get(url_noticias)

In [3]:
html

<Response [200]>

In [4]:
soup = BeautifulSoup(html.text, 'html.parser')

In [9]:
articles = soup.find_all('article', {'class': 'standard-teaser-container condensed-horizontal news'})

32

In [46]:
def get_artcl_data(article):
        try:
            if (article.find(name= "h5", attrs= {"class": "standard-teaser-subheading slug"}).text== "Seguridad"
                or article.find(name= "h5", attrs= {"class": "standard-teaser-subheading slug"}).text == "Noticias"):

                article_title= article.find(name= 'h3', attrs= {'class':'standard-teaser-headline teaser-headline'}).text
                article_path= article.find(name= "a", href= True)['href']
                article_url= "https://www.elnuevodia.com{path}".format(path= article_path)
                article_author= article.find("div", {"class": "authors-byline-text"}).text
                
                return {"Title": article_title, "url": article_url, "author": article_author}
        except AttributeError:
             print("¡No se encontró resultado!")

data_handler= {}

for artcl in zip(count(start= 1, step= 1), articles):

    artcl_data= get_artcl_data(article= artcl[1])

    data_handler[str(artcl[0])]= artcl_data


{'1': {'Title': 'Reportan un “carjacking” a plena luz del día en Río PiedrasUna conductora alegó haber sido asaltada por un sujeto armado a eso de las 10:30 de la mañana  ', 'url': 'https://www.elnuevodia.com/noticias/seguridad/notas/reportan-un-carjacking-a-plena-luz-del-dia-en-rio-piedras/', 'author': 'Por Agustín Criollo Oquero'}, '2': {'Title': 'Hallan un cuerpo amordazado y quemado en Vega BajaFue encontrado durante un patrullaje preventivo de la Policía', 'url': 'https://www.elnuevodia.com/noticias/seguridad/notas/hallan-un-cuerpo-amordazado-y-quemado-en-vega-baja/', 'author': 'Por Alex Figueroa Cancel'}, '3': {'Title': 'Hallan el cadáver de un hombre flotando a orillas de un río en ManatíEl cuerpo fue localizado debajo del puente Montebello en la PR-6685', 'url': 'https://www.elnuevodia.com/noticias/seguridad/notas/hallan-el-cadaver-de-un-hombre-flotando-a-orillas-de-un-rio-en-manati/', 'author': 'Por Andrea Guemárez Soto'}, '4': {'Title': 'Arrestan al presunto líder de un punto

In [47]:
note_url= data_handler["1"]["url"]


'https://www.elnuevodia.com/noticias/seguridad/notas/reportan-un-carjacking-a-plena-luz-del-dia-en-rio-piedras/'